In [ ]:
import os
import asyncio
from functools import lru_cache
from dotenv import load_dotenv, find_dotenv

import httpx
import pandas as pd

load_dotenv(find_dotenv())

In [ ]:
def create_client() -> httpx.AsyncClient:
    """
    Creates a common client for future http requests

    Returns:
        httpx.Client: client with ford proxies
    """
    limits = httpx.Limits(max_connections=8)  # test with 8 latter, default 4
    ford_proxy = httpx.Proxy(str(os.getenv("FORD_PROXY")))
    proxy_mounts = {
        "http://": httpx.AsyncHTTPTransport(proxy=ford_proxy, limits=limits, retries=3),
        "https://": httpx.AsyncHTTPTransport(
            proxy=ford_proxy, limits=limits, retries=3
        ),
    }
    return httpx.AsyncClient(
        timeout=httpx.Timeout(10.0, connect=5.0, pool=4.0),
        mounts=proxy_mounts,
        #        verify=False,
    )

In [ ]:
def get_token():
    with httpx.Client(
        proxies={
            "http://": str(os.getenv("FORD_PROXY")),
            "https://": str(os.getenv("FORD_PROXY")),
        }
    ) as client:
        response = client.post(
            str(os.getenv("TOKEN_ENDPOINT")),
            data={
                "client_id": str(os.getenv("CLIENT_ID")),
                "client_secret": str(os.getenv("CLIENT_SECRET")),
                "scope": str(os.getenv("SCOPE")),
                "grant_type": "client_credentials",
            },
            timeout=160,
        )
    print(
        "token expires in:",
        round(int(response.json()["expires_in"]) / 60, 0),
        "minutes",
    )
    return response.json()["access_token"]


def load_categories():
    categories = []
    with open(
        "../../../../data/external/binnings.txt", encoding="utf-8", mode="r"
    ) as file:
        for line in file.readlines():
            categories.append(line.strip("\n").split(",")[1])
    return frozenset(categories)


token = get_token()

In [ ]:
@lru_cache(maxsize=70)
async def call_api_movement(
    complaint: str, client: httpx.AsyncClient, token: str, semaphore: asyncio.Semaphore
):
    parts = (
        "door, window, windshield, wiper, glass, hood, trunk, moonroof, "
        + "bumper, tail light, pillar, undershield, roof rack, latch, he"
        + "adlight, door handle, door keypad, window, weatherstripping, "
        + "side mirror, lighting, swing gate, cowl grille, hard top, ski"
        + "d plate, sheet metal, running boards, water leak, etc)"
    )

    prompt = (
        f"{complaint}"
        "Question 1: For this complaint, check if it is related to an ex"
        + f"ternal part of the car, body exterior, ({parts}). If ye"

        + "s, answer 'F8'. Otherwise, answer 'NOT F8'. Note that most of"
        + "the problems related to power liftgate electrical problems an"
        + "d rear view camera are NOT F8. Question 2: For each of these "
        + "sentences that your answer 1 was 'F8', check if it is related"
        + f"to only one of the following categories: {list(load_categories())}."
        + " You should give only one answer with one answer for Question"
        + " 1 and one answer for Question 2 in the following format: 'AN"

        + "SWER 1~~~ANSWER 2'. Note: 'OWD' means 'opened while driving' "
        + "and 'F&F' means 'fit and finish', for problems related to flu"
        + "shness and margin. Note 2: For model Escape (2020 forward), t"
        + "here is a common problem related to door check arm when the c"
        + "omplaint is related to the door making popping sounds, openin"
        + "g and closing problens, hinges and welds. If you cannot relat"
        + "e, answer NOT SURE. Answer in the correct order. If you canno"
        + "t assist, answer 1, and answer 2 must be NA. You should be ob"
        + "jective and cold. Never change the answer format mentioned."
    )
    content = {
        "model": "gpt-4",
        "context": (
            "You are a helpful text reader and analyzer. You need to give me 2 answers."
        ),  # sets the overall behavior of the assistant.
        "messages": [{"role": "user", "content": prompt}],
        "parameters": {
            "temperature": 0.05,  # Determines the randomnes of the model's response.
        },
    }
    async with semaphore:
        response = await client.post(
            str(os.getenv("API_ENDPOINT")),
            headers={"Authorization": f"Bearer {token}"},
            json=content,
            timeout=8,
        )

    message = response.json()["content"]
    print(message)

    if len(parts := message.split("~~~")) == 2:
        function, result = parts

        if function == "NOT F8":
            return [function, "~", "~"]

        if function == "F8":
            if "|" not in result:  # case where recalls is being processed
                return [function, "~", result]
            component, failure = result.split(" | ")
            return [function, component, failure]

    return ["NOT CLASSIFIED", "~", "~"]

In [ ]:
df = pd.read_csv("../../../../data/raw/mock_dataset.csv")
async with create_client() as client:
    df[["FUNCTION_", "COMPONET", "FAILURE"]] = await asyncio.gather(
        *(
            call_api_movement(conmplaint, client, token, asyncio.Semaphore(70))
            for conmplaint in df["CDESCR"]
        )
    )
df.head()